In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

/Users/suchit/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
DataDir = "dataset"
categories = ["Beagle", "Boxer", "Bulldog", "Dachshund", "German_Shepherd", "Golden_Retriever", "Husky", "Labrador_Retriever", "Poodle", "Rottweiler", "Yorkshire_Terrier"]

training_data = []
img_size = 70

def create_training_data():
    for category in categories:
        path = os.path.join(DataDir, category)
        y_value = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                if img_array is not None:
                    new_array = cv2.resize(img_array, (img_size, img_size))
                    training_data.append([new_array, y_value])
                else:
                    print(f"Warning: Unable to read image {img} in category {category}")
            except Exception as e:
                print(f"Error processing image {img} in category {category}: {e}")

create_training_data()

# Shuffle the data to ensure a good mix of classes
import random
random.shuffle(training_data)

# Separate features and labels
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

# Convert to NumPy arrays and reshape
X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)

# Normalize the pixel values
X = X / 255.0

print("Data preparation complete.")
print(f"Total samples: {len(training_data)}")

libpng warning: iCCP: known incorrect sRGB profile


Data preparation complete.
Total samples: 1056


In [ ]:
import os

DataDir = "dataset"
categories = ["Beagle", "Boxer", "Bulldog", "Dachshund", "German_Shepherd", "Golden_Retriever", "Husky", "Labrador_Retriever", "Poodle", "Rottweiler", "Yorkshire_Terrier"]

def count_images_per_breed():
    breed_counts = {}
    for category in categories:
        path = os.path.join(DataDir, category)
        if os.path.exists(path):
            num_images = len([img for img in os.listdir(path) if os.path.isfile(os.path.join(path, img))])
            breed_counts[category] = num_images
        else:
            breed_counts[category] = 0
    return breed_counts

breed_counts = count_images_per_breed()

for breed, count in breed_counts.items():
    print(f"{breed}: {count} images")

In [9]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

In [10]:
num_classes = 11

if not isinstance(X, np.ndarray):
    X = np.array(X)
if not isinstance(y, np.ndarray):
    y = np.array(y)

y = tf.keras.utils.to_categorical(y, num_classes)

#building the model and adding the layers
model = Sequential()

# Layer 1
model.add(Conv2D(32, (3, 3), input_shape=(img_size, img_size, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Layer 3
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

log_dir = "logs/fit/" + time.strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(X, y, batch_size=32, validation_split=0.1, epochs=10, callbacks=[tensorboard_callback])

# Save the model
model.save('dog_breed_classifier.h5')

/Users/suchit/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - accuracy: 0.0767 - loss: 2.4149 - val_accuracy: 0.0755 - val_loss: 2.3976
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - accuracy: 0.1096 - loss: 2.3943 - val_accuracy: 0.1415 - val_loss: 2.3670
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.1773 - loss: 2.2967 - val_accuracy: 0.4151 - val_loss: 1.9700
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.3341 - loss: 1.9133 - val_accuracy: 0.5849 - val_loss: 1.4083
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.5319 - loss: 1.3281 - val_accuracy: 0.7925 - val_loss: 0.8498
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - accuracy: 0.7032 - loss: 0.8871 - val_accuracy: 0.8585 - val_loss: 0.5486
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.8044 - loss: 0.6628 - val_accuracy: 0.8868 - val_loss: 0.3651
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.8835 - loss: 0.3985 - val_accuracy: 0.9528 -

In [11]:
#predicting the image
from tensorflow.keras.models import load_model
model = load_model('dog_breed_classifier.h5')

def prepare_image(filepath, img_size=70):
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # Read the image in grayscale
    new_array = cv2.resize(img_array, (img_size, img_size))  # Resize the image to match the input shape
    new_array = new_array.reshape(-1, img_size, img_size, 1)  # Reshape the image to add batch dimension
    new_array = new_array / 255.0  # Normalize the image
    return new_array

def predict_breed(filepath):
    prepared_image = prepare_image(filepath)
    prediction = model.predict(prepared_image)
    breed_index = np.argmax(prediction)
    breed_name = categories[breed_index]
    return breed_name

In [ ]:
image_path = 'path_to_your_dog_image.jpg'
breed_name = predict_breed(image_path)
print(f"The predicted breed is: {breed_name}")